# Classification
<hr>

## 1. Binary Classification (XOR)

#### (1) Load Data

In [1]:
import torch

In [2]:
train_data = torch.tensor([[0.,0.], [0.,1.], [1.,0.], [1.,1.]], requires_grad=True)
targets = torch.tensor([0.,1.,1.,0.]).view(-1, 1)

#### <strike>(2) Define Dataloader</strike>

- Doesn't Need PyTorch Built-in Dataloader for Small Data
- Doesn't Need Custom Dataloader for No Preprocessing Required Data

#### (3) Define Model

In [3]:
import torch.nn as nn

In [4]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.fc1 = nn.Linear(2, 3)
        self.fc2 = nn.Linear(3, 1)
        
    def forward(self, x):
        x = torch.sigmoid(self.fc1(x))
        x = torch.sigmoid(self.fc2(x))
        return x
        
model = Net()

In [5]:
# check which layers consist Network
print(model)

Net(
  (fc1): Linear(in_features=2, out_features=3, bias=True)
  (fc2): Linear(in_features=3, out_features=1, bias=True)
)


#### (4) Set Loss & Optimizer

In [6]:
criterion = nn.MSELoss()
optimizer = torch.optim.SGD(model.parameters(), lr=0.05)

#### (5) Train / Test

In [7]:
import numpy as np # To Check Trained Parameters

In [8]:
epochs = 15000
for idx in range(epochs):
    for i, (input, target) in enumerate(zip(train_data, targets)):
        # Forward Propagation
        output = model(input)
        
        # Get Loss, Compute Gradient, Update Parameters
        loss = criterion(output, target)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        # Print Loss for Tracking Training
        if idx % 1000 == 0 and (idx//1000)%4 == i:
            print("Epoch: {: >8} | Loss: {:8f} | Output: {:4f} | Target: {}".format(idx, loss, output.data[0], target.data[0]))
            
# Test after Training is done
with torch.no_grad():
    print("-----------------------------------------------------------------")
    print("Trained Parameters:")
    for name, param in model.named_parameters():
        if param.requires_grad:
            print(name, param.data.size(), '\n', np.round_(param.data.numpy(),2))
            
    print("-----------------------------------------------------------------")
    print("Final results:")
    for input, target in zip(train_data, targets):
        output = model(input)
        print("Input: {} | Output: {:4f} | Target: {}".format(input.data, output.data[0], target.data[0]))

Epoch:        0 | Loss: 0.226407 | Output: 0.475823 | Target: 0.0
Epoch:     1000 | Loss: 0.259013 | Output: 0.491066 | Target: 1.0
Epoch:     2000 | Loss: 0.217400 | Output: 0.533738 | Target: 1.0
Epoch:     3000 | Loss: 0.387208 | Output: 0.622260 | Target: 0.0
Epoch:     4000 | Loss: 0.020469 | Output: 0.143071 | Target: 0.0
Epoch:     5000 | Loss: 0.174210 | Output: 0.582615 | Target: 1.0
Epoch:     6000 | Loss: 0.033172 | Output: 0.817868 | Target: 1.0
Epoch:     7000 | Loss: 0.023813 | Output: 0.154316 | Target: 0.0
Epoch:     8000 | Loss: 0.002924 | Output: 0.054078 | Target: 0.0
Epoch:     9000 | Loss: 0.005916 | Output: 0.923083 | Target: 1.0
Epoch:    10000 | Loss: 0.005290 | Output: 0.927269 | Target: 1.0
Epoch:    11000 | Loss: 0.005539 | Output: 0.074427 | Target: 0.0
Epoch:    12000 | Loss: 0.000795 | Output: 0.028188 | Target: 0.0
Epoch:    13000 | Loss: 0.002329 | Output: 0.951741 | Target: 1.0
Epoch:    14000 | Loss: 0.002536 | Output: 0.949638 | Target: 1.0
----------

<hr id="border1">

## 2. Multi-class Classification (MNIST)

#### (0) Define Hyper-parameters

In [9]:
import torch

In [10]:
# Device configuration
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Hyper-parameters
input_size = 784
hidden_size = 500
num_classes = 10
num_epochs = 5
batch_size = 100
learning_rate = 0.001

#### (1) Load Data

In [11]:
import torchvision
import torchvision.transforms as transforms

In [12]:
train_dataset = torchvision.datasets.MNIST(root='./datasets',
                                           train=True,
                                           transform=transforms.ToTensor(),
                                           download=True)

test_dataset = torchvision.datasets.MNIST(root='./datasets',
                                          train=False,
                                          transform=transforms.ToTensor())

In [13]:
# cf) check for the data
image, label = train_dataset[0]
print(len(train_dataset), image.size(), label)
image, label = test_dataset[0]
print(len(test_dataset), image.size(), label)

60000 torch.Size([1, 28, 28]) tensor(5)
10000 torch.Size([1, 28, 28]) tensor(7)


#### (2) Define Dataloader

In [14]:
data_loader = torch.utils.data.DataLoader(dataset=train_dataset,
                                           batch_size=batch_size,
                                           shuffle=True)

test_loader = torch.utils.data.DataLoader(dataset=test_dataset,
                                          batch_size=batch_size,
                                          shuffle=False)

In [15]:
# cf) check how data_loader works
print(len(data_loader))
for idx, (images, labels) in enumerate(data_loader):
    if idx == 3:
        break
    print(images.size(), labels)

600
torch.Size([100, 1, 28, 28]) tensor([3, 0, 6, 6, 9, 1, 9, 3, 8, 3, 3, 3, 8, 7, 1, 3, 6, 4, 2, 7, 8, 3, 2, 3,
        7, 5, 7, 8, 2, 2, 0, 8, 4, 1, 0, 6, 4, 1, 5, 0, 1, 0, 7, 4, 5, 2, 7, 0,
        8, 3, 3, 3, 4, 0, 8, 2, 4, 2, 5, 7, 1, 7, 3, 4, 2, 6, 6, 6, 0, 7, 7, 2,
        8, 2, 2, 0, 1, 0, 6, 0, 9, 6, 6, 9, 4, 5, 7, 6, 0, 3, 8, 0, 8, 9, 7, 3,
        1, 9, 8, 2])
torch.Size([100, 1, 28, 28]) tensor([5, 0, 4, 4, 1, 3, 3, 5, 6, 9, 8, 9, 1, 7, 1, 7, 8, 2, 5, 4, 6, 3, 8, 9,
        5, 9, 9, 3, 4, 4, 1, 2, 5, 0, 6, 6, 9, 3, 7, 0, 9, 5, 7, 9, 0, 0, 6, 1,
        1, 5, 9, 2, 3, 6, 4, 5, 0, 9, 3, 0, 3, 6, 9, 3, 5, 2, 1, 4, 9, 6, 9, 6,
        4, 4, 5, 1, 5, 1, 1, 4, 4, 4, 3, 3, 1, 9, 5, 1, 5, 0, 0, 5, 0, 2, 8, 4,
        1, 0, 7, 8])
torch.Size([100, 1, 28, 28]) tensor([1, 3, 0, 5, 8, 0, 3, 2, 2, 7, 9, 1, 0, 1, 9, 0, 2, 3, 5, 9, 8, 9, 1, 6,
        4, 0, 4, 3, 1, 0, 9, 3, 7, 9, 3, 3, 7, 2, 0, 8, 3, 3, 5, 8, 7, 6, 3, 4,
        9, 3, 5, 4, 9, 0, 7, 9, 9, 2, 1, 9, 8, 0, 7, 1, 1, 5, 2, 6,

#### (3) Define Model

In [16]:
import torch.nn as nn

In [17]:
class Net(nn.Module):
    def __init__(self, input_size, hidden_size, num_classes):
        super(Net, self).__init__()
        self.fc1 = nn.Linear(input_size, hidden_size)
        self.relu = nn.ReLU()
        self.fc2 = nn.Linear(hidden_size, num_classes)

    def forward(self, x):
        x = self.fc1(x)
        x = self.relu(x)
        x = self.fc2(x)
        return x

model = Net(input_size, hidden_size, num_classes).to(device)


In [18]:
# check which layers consist Network
print(model)

Net(
  (fc1): Linear(in_features=784, out_features=500, bias=True)
  (relu): ReLU()
  (fc2): Linear(in_features=500, out_features=10, bias=True)
)


In [19]:
# cf) check how data passes through the Network
data_iter = iter(data_loader)
images, labels = data_iter.next()
images = images.reshape(-1, 28*28) # reshape dimensions of the input images to fit model
outputs = model(images)
print(outputs.size(), '\n', outputs[0], '\n', outputs[0].tolist().index(max(outputs[0])))

torch.Size([100, 10]) 
 tensor([ 0.0378,  0.0070,  0.0120,  0.0289,  0.1047,  0.1198, -0.0292,  0.0338,
        -0.0481,  0.0369], grad_fn=<SelectBackward>) 
 5


#### (4) Set Loss & Optimizer

In [20]:
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

#### (5) Train / Test

In [21]:
for epoch in range(num_epochs):
    for i, (images, labels) in enumerate(data_loader):
        # Assign Tensors to Configured Device
        images = images.reshape(-1, 28*28).to(device) # reshape dimensions of the input images to fit model
        labels = labels.to(device)

        # Forward Propagation
        outputs = model(images)

        # Get Loss, Compute Gradient, Update Parameters
        loss = criterion(outputs, labels)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        # Print Loss for Tracking Training
        if (i+1) % 100 == 0:
            print ('Epoch [{}/{}], Step [{}/{}], Loss: {:.4f}'.format(epoch+1, num_epochs, i+1, len(data_loader), loss.item()))

# Test after Training is done
with torch.no_grad():
    correct = 0
    total = 0
    for images, labels in test_loader:
        images = images.reshape(-1, 28*28).to(device) # reshape dimensions of the input images to fit model
        labels = labels.to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

    print('Accuracy of the network on the 10000 test images: {} %'.format(100 * correct / total))

Epoch [1/5], Step [100/600], Loss: 0.4084
Epoch [1/5], Step [200/600], Loss: 0.2091
Epoch [1/5], Step [300/600], Loss: 0.2739
Epoch [1/5], Step [400/600], Loss: 0.3488
Epoch [1/5], Step [500/600], Loss: 0.2962
Epoch [1/5], Step [600/600], Loss: 0.0703
Epoch [2/5], Step [100/600], Loss: 0.0664
Epoch [2/5], Step [200/600], Loss: 0.1353
Epoch [2/5], Step [300/600], Loss: 0.1843
Epoch [2/5], Step [400/600], Loss: 0.0856
Epoch [2/5], Step [500/600], Loss: 0.1021
Epoch [2/5], Step [600/600], Loss: 0.1700
Epoch [3/5], Step [100/600], Loss: 0.0552
Epoch [3/5], Step [200/600], Loss: 0.0535
Epoch [3/5], Step [300/600], Loss: 0.0523
Epoch [3/5], Step [400/600], Loss: 0.1097
Epoch [3/5], Step [500/600], Loss: 0.0716
Epoch [3/5], Step [600/600], Loss: 0.1098
Epoch [4/5], Step [100/600], Loss: 0.0337
Epoch [4/5], Step [200/600], Loss: 0.1234
Epoch [4/5], Step [300/600], Loss: 0.0464
Epoch [4/5], Step [400/600], Loss: 0.0341
Epoch [4/5], Step [500/600], Loss: 0.0109
Epoch [4/5], Step [600/600], Loss:

#### (6) Save & Visualization

In [22]:
torch.save(model.state_dict(), 'model.ckpt')